In [0]:
file_cust = dbutils.fs.ls("/Volumes/bigdata/bdata/customer")
display(file_cust)

In [0]:
for i in file_cust:
    if '.parquet' in file_cust and file_cust.size > 1:
        df_cust = spark.read.format("parquet").load("/Volumes/bigdata/bdata/customer/*.parquet")
    else:
        df_cust = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/Volumes/bigdata/bdata/customer/*.csv")

df_cust.display()


df_cust.write.format("delta").mode("append").option("mergeSchema", True).saveAsTable('bigdata.bdata.customer_dlt1')


In [0]:
%sql
select * from bigdata.bdata.customer_dlt1

In [0]:
dbutils.fs.put('/Volumes/bigdata/bdata/customer/customer_ref2.csv', '''
Feedback_ID,Product,Rating,Comment,date
100000,Product1,5,Great Product,01/01/2021
100001,Product2,4,Good Product,03/05/2021
100002,Product3,3,Product is ok,06/03/2011''')

In [0]:
from pyspark.sql.functions import to_date, col

df_cust_ref = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/Volumes/bigdata/bdata/customer/customer_ref2.csv")
df_cust_ref.display()

df_cust_ref.write.format("delta").mode("append").option("mergeSchema", True).saveAsTable("bigdata.bdata.customer_dlt1")

In [0]:
from pyspark.sql.functions import col

df_cust_ref = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/Volumes/bigdata/bdata/customer/customer_ref2.csv")
)

df_cust_ref = (
    df_cust_ref
    .withColumn("Feedback_ID", col("Feedback_ID").cast("string"))
    .withColumn("Rating", col("Rating").cast("string"))
)

display(df_cust_ref)

df_cust_ref.write.format("delta").mode("append").option("mergeSchema", True).saveAsTable("bigdata.bdata.customer_dlt1")


In [0]:
%sql
select * from bigdata.bdata.customer_dlt1 
where date is not null and comment = 'Great Product'
order By Feedback_ID desc;

In [0]:
from pyspark.sql.functions import to_date, col

df_cust_ref = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/Volumes/bigdata/bdata/customer/customer_ref1.csv")
df_cust_date = df_cust_ref.withColumn("date", to_date(col("date"), "dd/MM/yyyy"))
df_cust_date.display()

df_cust_date.write.format("delta").mode("overwrite").option("mergeSchema", True).saveAsTable('bigdata.bdata.customer_dlt')